In [1]:
import os
import re
import numpy as np
import pandas as pd
from charset_normalizer import from_path

In [4]:
# function to extract number of words in scene
def extract_scene_lengths(file_path):
    result = from_path(file_path).best()
    with open(file_path, 'r', encoding=result.encoding) as file:
        lines = file.readlines()
    text = ''.join(lines[1:])
    scenes = text.split('=' * 50)
    scene_lengths = [len(scene.strip().split()) for scene in scenes if scene.strip()]
    return scene_lengths

# function to get mean length of scenes and standard deviation from mean
def analyze_scene_lengths(scene_lengths):
    mean_length = np.mean(scene_lengths)
    std_length = np.std(scene_lengths)
    return mean_length, std_length

# function to calculate coefficient of variation
def coherence_classifier(mean_length, std_length):
    coefficient_of_variation = std_length / mean_length
    return coefficient_of_variation

# function to process all screenplays
def process_scene_lengths(folder_path):
    coefficients_of_variation = {}
    
    # get all .txt files in folder
    files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

    for filename in files:
        file_path = os.path.join(folder_path, filename)
        
        # extract scene lengths
        scene_lengths = extract_scene_lengths(file_path)
        
        if scene_lengths:
            # analyze scene lengths
            mean_length, std_length = analyze_scene_lengths(scene_lengths)
            coefficient_of_variation = coherence_classifier(mean_length, std_length)
            
            # check if filename matches pattern
            match = re.search(r'_0*(\d+)\.txt$', filename)
            if match:
                # extract imdbid from filename
                imdbid = int(match.group(1))
                coefficients_of_variation[imdbid] = coefficient_of_variation
    
    return coefficients_of_variation

In [5]:
# read csv file
df = pd.read_csv('data/01_movie_metadata.csv')

# folder containing screenplay files
screenplay_folder = 'data/screenplay_data/data/scene_separated_texts'

# process scene lengths and get coefficients of variation
coefficients_of_variation = process_scene_lengths(screenplay_folder)

# add new column for scene length coefficient of variation
df['scene_length_cv'] = df['imdbid'].map(coefficients_of_variation)

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna(subset=['scene_length_cv'])
df.shape

In [ ]:
# save updated dataframe
df.to_csv('data/02_movie_metadata.csv', index=False)